# Pickle

In [2]:
import pickle

my_object = {"test": "bom teste"}

In [9]:
filename = "my_obj.pickle"
pickle.dump(my_object, open(filename, "wb"))

In [10]:
my_obj_aloaded =  pickle.load(open(filename, 'rb'))

In [11]:
my_obj_loaded

{'test': 'bom teste'}

# Label Encoder

In [22]:
import pandas as pd
df = pd.DataFrame({"col1": ["A", "A", "B"], "col2": ["H", "M", "G"], "target": [1, 1, 0]})
df

,col1,col2,target
0,A,H,1
1,A,M,1
2,B,G,0


In [15]:
from sklearn.preprocessing import LabelEncoder

label_encoders = {}

for col in df.columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

In [16]:
filename = "label_encoders.pickle"
pickle.dump(label_encoders, open(filename, "wb"))

Carregando para aplicar na base de produção:

In [17]:
encoders =  pickle.load(open(filename, 'rb'))

In [19]:
df_prod = pd.DataFrame({"col1": ["A", "A", "B", "B", "B"], "col2": ["H", "M", "G", "G", "M"]})

In [20]:
for col in df_prod.columns:
    df_prod[col] = encoders[col].transform(df_prod[col])
    
df_prod

,col1,col2
0,0,1
1,0,2
2,1,0
3,1,0
4,1,2


# Pipelines

In [44]:
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

class ApplyLabelEncoder():
    
    def __init__(self):
        self.label_dict = {}
        
    def fit_transform(self, X, *args, **kwargs):
        for col in X.dtypes[X.dtypes == "object"].index.values:
            le = LabelEncoder()
            X[col] = le.fit_transform(X[col])
            self.label_dict[col] = le
        return X
            
    def transform(self, X, *args, **kwargs):
        for col in X.dtypes[X.dtypes == "object"].index.values:
            X[col] = self.label_dict[col].transform(X[col])
        return X
        
    
    
features = ["col1", "col2"]
target = "target"
pipe = Pipeline([("le", ApplyLabelEncoder()), ("model", DecisionTreeClassifier())])
pipe = pipe.fit(df[features], df[target])

/Users/lgmoneda/miniconda2/envs/fia/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [46]:
pipe.predict_proba(df_prod[features])

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [47]:
filename = "my_pipe.pickle"
pickle.dump(pipe, open(filename, "wb"))

In [48]:
my_pipe = pickle.load(open(filename, 'rb'))

In [49]:
my_pipe.predict_proba(df_prod[features])

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.]])